In [1]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
import rdkit
import os 

import matplotlib.pyplot as plt

from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import (LeaveOneOut, cross_val_predict,
                                     cross_val_score, train_test_split)

import numpy as np 
import re

from rdkit.Chem import Descriptors
#print(len(Descriptors._descList))
#print(Descriptors._descList[:5])
#print(rdkit.__version__)

from contextlib import contextmanager,redirect_stderr,redirect_stdout
from os import devnull

# build correlation matrix for the descriptors
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def get_top_correlations_blog(df, threshold=0.4):
    """
    df: the dataframe to get correlations from
    threshold: the maximum and minimum value to include for correlations. For eg, if this is 0.4, only pairs haveing a correlation coefficient greater than 0.4 or less than -0.4 will be included in the results. 
    """
    orig_corr = df.corr()
    c = orig_corr.abs()

    so = c.unstack()

    #print("|    Variable 1    |    Variable 2    | Correlation Coefficient    |")
    #print("|------------------|------------------|----------------------------|")
    
    #i=0
    pairs=set()
    result = []
    for index, value in so.sort_values(ascending=False).items():
        # Exclude duplicates and self-correlations
        if value > threshold \
        and index[0] != index[1] \
        and (index[0], index[1]) not in pairs \
        and (index[1], index[0]) not in pairs:
            
            #print(f'|    {index[0]}    |    {index[1]}    |    {orig_corr.loc[(index[0], index[1])]}    |')
            #result.loc[i, ['Variable 1', 'Variable 2', 'Correlation Coefficient']] = \
            #    [index[0], index[1], orig_corr.loc[(index[0], index[1])]]
            result.append([index[0], index[1], orig_corr.loc[(index[0], index[1])]])
            pairs.add((index[0], index[1]))
            #i+=1
            
    #return result.reset_index(drop=True).set_index(['Variable 1', 'Variable 2'])
    return result

@contextmanager
def suppress_stdout_stderr():
    """A context manager that redirects stdout and stderr to devnull"""
    with open(devnull, 'w') as fnull:
        with redirect_stderr(fnull) as err, redirect_stdout(fnull) as out:
            yield (err, out)


def build_features_matrix_and_labels (molnames, descriptors, labels):
    # build features matrix and labels
    moldescriptors_featues = []
    Y = []
    features_names = []

    for idx, _ in enumerate(molnames):
        val = []
        for k,v in descriptors[idx].items():
            if idx == 0:
                features_names.append(k)
            val.append(v)
        moldescriptors_featues.append(val)
        Y.append(labels[idx])

    Y = np.array(Y)
    moldescriptors_featues = np.array(moldescriptors_featues)

    return  moldescriptors_featues, Y, features_names

DEfine basic data structures and config

In [2]:
# those are the output data
molnames = []
labels = []
diffs_toothermethods = []
chemical_reacts = []
stechio_ceofs = []
moldescriptors = []

howmanydifs = 3
rootdirqdata = '../datasets/qdata/'
rootdirdata = '../datasets/data/'

hflist = ["Nuclear Repulsion  :", \
          "One Electron Energy:", \
          "Two Electron Energy:", \
          "Potential Energy   :", \
          "Kinetic Energy     :", \
          "Dispersion correction", \
          "Total Charge", \
          "Multiplicity", \
          "Number of Electrons", \
          "FINAL SINGLE POINT ENERGY"]

pbelist = ["Nuclear Repulsion  :", \
            "One Electron Energy:", \
            "Two Electron Energy:", \
            "Potential Energy   :", \
            "Kinetic Energy     :", \
            "E(X)               :"  , \
            "E(C)               :"  , \
            "Dispersion correction", \
            "Total Charge"   , \
            "Multiplicity"   , \
            "Number of Electrons", \
            "FINAL SINGLE POINT ENERGY"]


Read molecules labels and more

In [3]:

fp = open(rootdirdata + '/labels.txt', 'r')

for line in fp:
    sline = line.replace("\t", " ").replace("\n", "").rstrip().lstrip().split()
    molname = sline[1]
    
    difvals = []   
    for i in range(howmanydifs):
        difvals.append(float(sline[-1*(i+1)]))

    schechio = []
    reacts = []
    for i in range(2,len(sline)-howmanydifs-1):
        nospace = sline[i].replace(" ", "")
        if nospace.isdigit():
            schechio.append(int(nospace))
        elif nospace.startswith("-") and nospace[1:].isdigit():
            schechio.append(int(nospace))
        else:
            reacts.append(nospace)

    stechio_ceofs.append(schechio)
    chemical_reacts.append(reacts)
    diffs_toothermethods.append(difvals)
    labels.append(float(sline[-1*howmanydifs-1]))
    molnames.append(molname)
    moldescriptors.append({})

fp.close()

print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))


Number of labels         : 140
Number of names          : 140
Number of differences    : 140
Number of chemicals      : 140
Number of stechio ceofs  : 140
Number of moldescriptors : 140


Read PBE data

In [4]:
pbedescriptor = {}

for file in os.listdir(rootdirqdata+'/PBE/'):
    if file.endswith('.out'):
        molname = file.split('.out')[0]
        molname = re.split("\.mpi\d+", molname)[0]
        #print(molname)
        moldesc = {}
        fp = open(rootdirqdata+'/PBE/'+file, 'r')
        for line in fp:
            for val in pbelist:
                if line.find(val) != -1:
                    keyval = val.replace(":", "").rstrip().lstrip().replace(" ", "_")
                    sline = line.rstrip().lstrip().split()
                    for sval in sline:
                        try:
                            firstnumvalue = float(sval)
                            break
                        except:
                            continue
                    
                    moldesc["PBE_"+keyval] = firstnumvalue
                    #print(molname, keyval, sval)
        fp.close()
        pbedescriptor[molname] = moldesc

for i, molname in enumerate(molnames):
    if molname in pbedescriptor:
        for k in pbedescriptor[molname].keys():
            moldescriptors[i][k] = pbedescriptor[molname][k]
    else:
        print(molname + " not found in PBE descriptors")

print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))


Number of labels         : 140
Number of names          : 140
Number of differences    : 140
Number of chemicals      : 140
Number of stechio ceofs  : 140
Number of moldescriptors : 140


Read HF data

In [5]:
hfdescriptor = {}

for file in os.listdir(rootdirqdata+'/HF/'):
    if file.endswith('.out'):
        molname = file.split('.out')[0]
        molname = re.split("\.mpi\d+", molname)[0]
        #print(molname)
        moldesc = {}
        fp = open(rootdirqdata+'/HF/'+file, 'r')
        for line in fp:
            for val in hflist:
                if line.find(val) != -1:
                    keyval = val.replace(":", "").rstrip().lstrip().replace(" ", "_")
                    sline = line.rstrip().lstrip().split()
                    for sval in sline:
                        try:
                            firstnumvalue = float(sval)
                            break
                        except:
                            continue
                    
                    moldesc["HF_"+keyval] = firstnumvalue
                    #print(molname, keyval, sval)
        fp.close()
        hfdescriptor[molname] = moldesc

for i, molname in enumerate(molnames):
    if molname in pbedescriptor:
        for k in hfdescriptor[molname].keys():
            moldescriptors[i][k] = hfdescriptor[molname][k]
    else:
        print(molname + " not found in HF descriptors")


print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))


Number of labels         : 140
Number of names          : 140
Number of differences    : 140
Number of chemicals      : 140
Number of stechio ceofs  : 140
Number of moldescriptors : 140


Remove molecules with some missing descriptor

In [6]:
alldims = set([len(val) for val in moldescriptors])
idxtoremovs = []
for i, val in enumerate(moldescriptors):
    if len(val) != max(alldims):
        idxtoremovs.append(i)

for i in sorted(idxtoremovs, reverse=True):
    print("Molname to remove:", molnames[i], "index:", i)
    del moldescriptors[i]
    del labels[i]
    del molnames[i]
    del diffs_toothermethods[i]
    del chemical_reacts[i]
    del stechio_ceofs[i]

print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))


Number of labels         : 140
Number of names          : 140
Number of differences    : 140
Number of chemicals      : 140
Number of stechio ceofs  : 140
Number of moldescriptors : 140


Remove molecules with None Label 

In [7]:
for i, v in enumerate(labels):
    if v is None:
        print("None value found in labels:", i, molnames[i])
        del moldescriptors[i]
        del labels[i]
        del molnames[i]
        del diffs_toothermethods[i]
        del chemical_reacts[i]
        del stechio_ceofs[i]

print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))


Number of labels         : 140
Number of names          : 140
Number of differences    : 140
Number of chemicals      : 140
Number of stechio ceofs  : 140
Number of moldescriptors : 140


If a descriptor is nan at least for a molecule remove from all

In [8]:
import math 

nandescriptors = set()
for index, molname in enumerate(molnames):
    if any(math.isnan(val) for val in moldescriptors[index].values()):
        print("Nan value found in descriptors:", molname)
        for k,v in moldescriptors[index].items():
            if math.isnan(v):
                nandescriptors.add(k)

print("Removing the following Descriptors ", nandescriptors)
print("Removing ", len(nandescriptors), " descriptors")
for i, v in enumerate(moldescriptors):
    for k in nandescriptors:
        del moldescriptors[i][k]


print("")
print("Number of descriptors:", len(moldescriptors[0]))
print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))

Removing the following Descriptors  set()
Removing  0  descriptors

Number of descriptors: 22

Number of labels         : 140
Number of names          : 140
Number of differences    : 140
Number of chemicals      : 140
Number of stechio ceofs  : 140
Number of moldescriptors : 140


Read chemicals

In [9]:
chemicals_descriptors = {}

for v in chemical_reacts:
    for chem in v:

        moldesc = {}
        fp = open(rootdirqdata + '/PBE/'+chem+'.out', 'r')
        for line in fp:
            for val in pbelist:
                if line.find(val) != -1:
                    keyval = val.replace(":", "").rstrip().lstrip().replace(" ", "_")
                    sline = line.rstrip().lstrip().split()
                    for sval in sline:
                        try:
                            firstnumvalue = float(sval)
                            break
                        except:
                            continue
                    
                    moldesc["PBE_"+keyval] = firstnumvalue

        fp.close

        fp = open(rootdirqdata + '/HF/'+chem+'.out', 'r')
        for line in fp:
            for val in hflist:
                if line.find(val) != -1:
                    keyval = val.replace(":", "").rstrip().lstrip().replace(" ", "_")
                    sline = line.rstrip().lstrip().split()
                    for sval in sline:
                        try:
                            firstnumvalue = float(sval)
                            break
                        except:
                            continue
                    
                    moldesc["HF_"+keyval] = firstnumvalue
        fp.close()

        chemicals_descriptors[chem] = moldesc

print("")
print("Number of chemicals descriptors:", len(chemicals_descriptors))


Number of chemicals descriptors: 12


Check error respect to QM methods

In [10]:

for methodid in range(howmanydifs):
    y_pred = []
    for i, molname in enumerate(molnames):
        y_pred.append(labels[i] + diffs_toothermethods[i][methodid])

    # plot a scatterplot of the true vs predicted values
    #plt.figure(figsize=(10,10))
    #plt.scatter(labels, y_pred, c='crimson')
    #plt.xlabel('True')
    #plt.ylabel('Predicted Values')
    
    print("Method", methodid+1, "R2 score  :", r2_score(labels, y_pred))
    print("Method", methodid+1, "RMSE score:", mean_squared_error(labels, y_pred, squared=False))

y_pred = []
autokcalmol = 627.5096080305927
for mi, molname in enumerate(molnames):
    #print(molname)
    oury = moldescriptors[mi]["PBE_FINAL_SINGLE_POINT_ENERGY"]
    si = 1
    tosub = 0.0
    for ci, chem in enumerate(chemical_reacts[mi]):
        stecchio = stechio_ceofs[mi][si]
        tosub += stecchio*chemicals_descriptors[chem]["PBE_FINAL_SINGLE_POINT_ENERGY"]
        si += 1
        
    y_pred.append(autokcalmol*(tosub-oury))
    #print(molname, oury, tosub, 627.51* (tosub-oury), labels[mi])

# plot a scatterplot of the true vs predicted values
#plt.figure(figsize=(10,10))
#plt.scatter(labels, y_pred, c='crimson')
#plt.xlabel('True')
#plt.ylabel('Predicted Values')

print("Our Method R2 score  :", r2_score(labels, y_pred))
print("Our Method RMSE score:", mean_squared_error(labels, y_pred, squared=False))  
     

Method 1 R2 score  : 0.9899437365837602
Method 1 RMSE score: 19.259720069171752
Method 2 R2 score  : 0.9905892336475421
Method 2 RMSE score: 18.631342333360124
Method 3 R2 score  : 0.990716922542401
Method 3 RMSE score: 18.50451203664354
Our Method R2 score  : 0.9228150294408229
Our Method RMSE score: 53.3578131123409


Build PBE and HF differences and thus descriptors

In [11]:
import models
%reload_ext models

pbe_hf_nonenergy_descriptors = []
pbe_diff_energy_descriptors = []
hf_diff_energy_descriptors = []

pbeenergylist = ["PBE_Nuclear_Repulsion", \
            "PBE_One_Electron_Energy", \
            "PBE_Two_Electron_Energy", \
            "PBE_Potential_Energy", \
            "PBE_Kinetic_Energy", \
            "PBE_E(X)"  , \
            "PBE_E(C)"  , \
            "PBE_Dispersion_correction", \
            "PBE_FINAL_SINGLE_POINT_ENERGY"]

hfenergylist = ["HF_Nuclear_Repulsion", \
          "HF_One_Electron_Energy", \
          "HF_Two_Electron_Energy", \
          "HF_Potential_Energy", \
          "HF_Kinetic_Energy", \
          "HF_Dispersion_correction", \
          "HF_FINAL_SINGLE_POINT_ENERGY"]

for mi, molname in enumerate(molnames):
    diff_desc = {}
    for desc in hfenergylist:
        y = moldescriptors[mi][desc]
        si = 1
        comp = 0.0
        for ci, chem in enumerate(chemical_reacts[mi]):
            stecchio = stechio_ceofs[mi][si]
            comp += stecchio*chemicals_descriptors[chem][desc]
            si += 1
        
        diff_desc[desc] = comp-y
    
    hf_diff_energy_descriptors.append(diff_desc)

for mi, molname in enumerate(molnames):
    diff_desc = {}
    for desc in pbeenergylist:
        y = moldescriptors[mi][desc]
        si = 1
        comp = 0.0
        for ci, chem in enumerate(chemical_reacts[mi]):
            stecchio = stechio_ceofs[mi][si]
            comp += stecchio*chemicals_descriptors[chem][desc]
            si += 1
        
        diff_desc[desc] = comp-y
    
    pbe_diff_energy_descriptors.append(diff_desc)

pbenonenergylist = ["PBE_Total_Charge", \
            "PBE_Multiplicity", \
            "PBE_Number_of_Electrons"]

hfnonenergylist = ["HF_Total_Charge", \
            "HF_Multiplicity", \
            "HF_Number_of_Electrons"]

if (len(pbenonenergylist) != len(hfnonenergylist)):
    print("Error: len(hfnonenergylist) != len(pbenonenergylist)")
    exit(1) 

# check if they are equal 
for mi, molname in enumerate(molnames):
    nondiff_desc = {}

    for idx in range(len(pbenonenergylist)):

        pbe_desc = pbenonenergylist[idx]
        hf_desc = hfnonenergylist[idx]  
        diff = moldescriptors[mi][pbe_desc] - moldescriptors[mi][hf_desc]

        if diff != 0.0:
            print("Error: diff != 0.0")
            print("molname:", molname)
            exit(1)
        else:  
            basidescname = pbe_desc.replace("PBE_", "").replace("HF_", "")
            nondiff_desc[basidescname] = moldescriptors[mi][pbe_desc]

    pbe_hf_nonenergy_descriptors.append(nondiff_desc)

print("")
print("Number of labels                       :", len(labels))
print("Number of names                        :", len(molnames))
print("Number of differences                  :", len(diffs_toothermethods))
print("Number of chemicals                    :", len(chemical_reacts))
print("Number of stechio ceofs                :", len(stechio_ceofs))
print("Number of moldescriptors               :", len(moldescriptors))
print("Number of pbe_hf_nonenergy_descriptors :", len(pbe_hf_nonenergy_descriptors))
print("Number of hf_diff_descriptors          :", len(hf_diff_energy_descriptors))
print("Number of pbe_diff_descriptors         :", len(pbe_diff_energy_descriptors))

2023-11-22 16:47:47.783046: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-22 16:47:47.783060: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



Number of labels                       : 140
Number of names                        : 140
Number of differences                  : 140
Number of chemicals                    : 140
Number of stechio ceofs                : 140
Number of moldescriptors               : 140
Number of pbe_hf_nonenergy_descriptors : 140
Number of hf_diff_descriptors          : 140
Number of pbe_diff_descriptors         : 140


Some correlation

In [12]:
fulldescriptors =[]

for idx, _ in enumerate(molnames):
    fulldescriptors.append({})
    fulldescriptors[idx].update(pbe_hf_nonenergy_descriptors[idx])
    fulldescriptors[idx].update(hf_diff_energy_descriptors[idx])
    fulldescriptors[idx].update(pbe_diff_energy_descriptors[idx])

moldescriptors_featues, Y, features_names = \
    build_features_matrix_and_labels (molnames, fulldescriptors, labels)

df = pd.DataFrame(moldescriptors_featues, columns=features_names)

top_corr = get_top_correlations_blog(df, 0.8)
for tc in top_corr:
    print(tc)


['PBE_Nuclear_Repulsion', 'HF_Nuclear_Repulsion', 1.0]
['HF_One_Electron_Energy', 'PBE_One_Electron_Energy', 0.9999980337156749]
['HF_Two_Electron_Energy', 'PBE_Two_Electron_Energy', 0.9999926955062333]
['PBE_Nuclear_Repulsion', 'PBE_One_Electron_Energy', -0.99995447781551]
['PBE_One_Electron_Energy', 'HF_Nuclear_Repulsion', -0.99995447781551]
['HF_One_Electron_Energy', 'HF_Two_Electron_Energy', -0.9999490337191265]
['HF_One_Electron_Energy', 'PBE_Nuclear_Repulsion', -0.9999485800751742]
['HF_One_Electron_Energy', 'HF_Nuclear_Repulsion', -0.9999485800751741]
['HF_One_Electron_Energy', 'PBE_Two_Electron_Energy', -0.9999442982407728]
['PBE_Two_Electron_Energy', 'PBE_One_Electron_Energy', -0.9999407994582038]
['PBE_One_Electron_Energy', 'HF_Two_Electron_Energy', -0.9999388037628172]
['PBE_Nuclear_Repulsion', 'HF_Two_Electron_Energy', 0.9998098786382636]
['HF_Two_Electron_Energy', 'HF_Nuclear_Repulsion', 0.9998098786382636]
['PBE_Two_Electron_Energy', 'HF_Nuclear_Repulsion', 0.999809706596

PLS models 

In [32]:

def run_all_pls (moldescriptors_featues, Y):
    maxcomp = moldescriptors_featues.shape[1]
    # search fo the best number od components and build final model
    perc_split = 0.2
    ncomps, rmses_test, rmses_train, r2s_test, r2s_train = \
        models.pls_model (0.2, moldescriptors_featues, Y, \
                      ncomp_start = 1, ncomp_max = maxcomp)
    r2max_comps = np.argmax(r2s_test)+1
    rmsemin_comps = np.argmin(rmses_test)+1
    compstouse = min(rmsemin_comps, r2max_comps)

    perc_split = 0.2
    rmse_train, rmse_test, r2_train, r2_test, rmse_full, r2_full  = \
        models.pls_model (0.2, moldescriptors_featues, Y, False, compstouse)
    perc_split = 0.0
    rmse, r2 = models.pls_model (perc_split, moldescriptors_featues, Y, False, \
                  compstouse, leaveoneout=True)
    
    return compstouse, rmse, r2, rmse_full, r2_full, rmse_test, r2_test, rmse_train, r2_train

fulldescriptors =[]
for idx, _ in enumerate(molnames):
    fulldescriptors.append({})
    fulldescriptors[idx].update(pbe_hf_nonenergy_descriptors[idx])
    fulldescriptors[idx].update(hf_diff_energy_descriptors[idx])
    fulldescriptors[idx].update(pbe_diff_energy_descriptors[idx])

fullenergydescriptors = {}
for idx, _ in enumerate(molnames):
    fullenergydescriptors[molnames[idx]] = {}
    fullenergydescriptors[molnames[idx]].update(hf_diff_energy_descriptors[idx])
    fullenergydescriptors[molnames[idx]].update(pbe_diff_energy_descriptors[idx])

for desctouse in  [pbe_diff_energy_descriptors, hf_diff_energy_descriptors, \
                   fulldescriptors, fullenergydescriptors]:
    moldescriptors_featues, Y, features_names = \
        build_features_matrix_and_labels (molnames, desctouse , labels)
    compstouse, rmse, r2, rmse_full, r2_full, rmse_test, r2_test, rmse_train, r2_train = \
        run_all_pls (moldescriptors_featues, Y) 
    print (compstouse, rmse, r2, rmse_full, r2_full, rmse_test, r2_test, rmse_train, r2_train )


9 16.002136580183628 0.9930578715461059 14.511900531819936 0.9942906681645354 17.034485386834895 0.9905871760598016 13.80944054653272 0.9950247044503004
5 56.71264575840334 0.9128040092517211 53.82537723227853 0.9214563889412524 52.99382611013372 0.9089012427955675 54.03126540530572 0.9238347288112724
17 14.293453992700144 0.9944612587630526 12.426516962497725 0.995813650010313 13.865149755163335 0.9937639293193127 12.040029499165612 0.99621799745156


KeyError: 0